# Names 
Spencer Bernardo-Cheng
Max Goldstein
Robbie Keehan

# Data Preparation

In [15]:
import numpy as np
import pandas as pd
datafb = pd.read_csv("./FB.csv")
datagoogle = pd.read_csv("./GOOGL.csv")
datamsft = pd.read_csv("./MSFT.csv")
dataam = pd.read_csv("./AMZN (1).csv")
datanflx = pd.read_csv("./NFLX.csv")

del datafb['Date']
del datagoogle['Date']
del datamsft['Date']
del datanflx['Date']


data = pd.concat([datafb, datagoogle, datamsft,datanflx, dataam ], axis=1, sort=False)
data = data[90:]
del data['Change']
del data['Gain']
del data['Loss']
del data['Average Gain']
del data['Average Loss']
del data['RS']
del data['DM+1']
del data['DM-1']
del data['TR-14']
del data['DM+1-14']
del data['DM-1+14']
del data['dl+1-14']
del data['dl-1-14']
del data['dl14diff']
del data['dl14sum']
del data['DX']
del data['TR']
del data['Date']

predictedDays = 14
up = .03
down =-.03
## Time difference the data
colheaders = data.columns[0:-3]

for header in colheaders: 
    for i in range(1, len(data)):
        data[header].at[90+i-1]= ((data[header].iloc[i]) - (data[header].iloc[i - 1]))/100

ydata = data['Close-amzn']
# ydata = ydata.apply(lambda x:2 if x>=up else (1 if (x< up and x > down) else 0))
ydata = pd.DataFrame(ydata)
ydata['newy'] = ""
# created target data in format to match output of multiple timesteps of 14 days ahead
# [0,1,2,3,4,5,6,7]
# [1,2,3,4,5,6,7,8]
for index, row in ydata.iterrows():
    temp = np.array(ydata['Close-amzn'][index-90:index-90+predictedDays].values)
    ydata['newy'].at[index] = temp
    
# removed last 14 because their targets are not able to created
ydata = ydata[0:-(predictedDays-1)]
data = data[0:-(predictedDays-1)]
del ydata['Close-amzn']


        
data = data[0:-1]
ydata = ydata[0:-1]
# data = data.T
print(ydata)



                                                   newy
90    [-0.05970002000000022, -0.007700040000000002, ...
91    [-0.007700040000000002, -0.018599999999999853,...
92    [-0.018599999999999853, 0.018199920000000418, ...
93    [0.018199920000000418, -0.002399910000000318, ...
94    [-0.002399910000000318, 0.03550002999999975, -...
...                                                 ...
1873  [0.06349976000000197, -0.2489001499999995, 0.0...
1874  [-0.2489001499999995, 0.014899910000001454, -0...
1875  [0.014899910000001454, -0.1510998600000039, 0....
1876  [-0.1510998600000039, 0.13040039000000206, 0.0...
1877  [0.13040039000000206, 0.0026000999999996567, -...

[1788 rows x 1 columns]


# One Hot Encoding The Target Data

In [9]:
from keras.utils.np_utils import to_categorical
print(ydata.head())
# ydata['newy'] = ydata['newy'].apply(lambda x: to_categorical(x,3))
print(ydata)

Using TensorFlow backend.


                                                 newy
90  [-0.05970002000000022, -0.007700040000000002, ...
91  [-0.007700040000000002, -0.018599999999999853,...
92  [-0.018599999999999853, 0.018199920000000418, ...
93  [0.018199920000000418, -0.002399910000000318, ...
94  [-0.002399910000000318, 0.03550002999999975, -...
                                                   newy
90    [-0.05970002000000022, -0.007700040000000002, ...
91    [-0.007700040000000002, -0.018599999999999853,...
92    [-0.018599999999999853, 0.018199920000000418, ...
93    [0.018199920000000418, -0.002399910000000318, ...
94    [-0.002399910000000318, 0.03550002999999975, -...
...                                                 ...
1873  [0.06349976000000197, -0.2489001499999995, 0.0...
1874  [-0.2489001499999995, 0.014899910000001454, -0...
1875  [0.014899910000001454, -0.1510998600000039, 0....
1876  [-0.1510998600000039, 0.13040039000000206, 0.0...
1877  [0.13040039000000206, 0.0026000999999996567, -...

[17

# Min-Max Scaling the Time Series

In [11]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(data.iloc[:,:].values)
print(x_scaled)



[[0.6704866  0.63964878 0.66867271 ... 0.31321402 0.35811382 0.15022488]
 [0.67729607 0.65028702 0.68004011 ... 0.34297397 0.26414926 0.12331708]
 [0.67596742 0.64724754 0.67519221 ... 0.42513468 0.25300192 0.10448737]
 ...
 [0.64872934 0.62174942 0.65580059 ... 0.38193301 0.3029723  0.1344608 ]
 [0.7002159  0.67139486 0.709629   ... 0.44102474 0.31038595 0.1153046 ]
 [0.68310912 0.70618038 0.67184883 ... 0.40337188 0.25243424 0.13533161]]


# Utilizing Time Series Generator

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 2
series = np.array(x_scaled)
target = np.array(ydata)
generator = TimeseriesGenerator(series, target, length=90, batch_size=1)
xtime = []
ytarget =[]
for i in range(len(generator)):
    x, y = generator[i]
    xtime.append(x[0])
    ytarget.append(y[0][0])
xtime = np.array(xtime)
ytarget = np.array(ytarget)




print(ytarget)

[[-0.00519989 -0.05550018 -0.0051001  ... -0.02950012  0.02890015
   0.00970001]
 [-0.05550018 -0.0051001   0.03890015 ...  0.02890015  0.00970001
  -0.09360016]
 [-0.0051001   0.03890015  0.01019989 ...  0.00970001 -0.09360016
  -0.03919983]
 ...
 [ 0.01489991 -0.15109986  0.13040039 ... -0.19200073 -0.11640015
  -0.0927002 ]
 [-0.15109986  0.13040039  0.0026001  ... -0.11640015 -0.0927002
  -0.20209961]
 [ 0.13040039  0.0026001  -0.0726001  ... -0.0927002  -0.20209961
   0.11119996]]


# Time Series Fold Splits

In [22]:
from sklearn.model_selection import train_test_split
import keras.utils
from keras import utils as np_utils
x_train,x_test,y_train,y_test = train_test_split(xtime, ytarget, test_size = 1/4, random_state = 1)

print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

y_train = y_train.reshape(1273,14,1)
y_test= y_test.reshape(425,14,1)



(1273, 90, 36)
(1273, 14)
(425, 14)


In [ ]:

model = Sequential()
model.add(LSTM(100, dropout=.2,activation='relu', input_shape=(90, 36), return_sequences=True))
model.add(Lambda(lambda x: x[:, -predictedDays:, :]))
model.add(Dense(1))

model.summary()
model.compile(optimizer='rmsprop', loss='mse')

# fit model
model.fit(x_train, y_train, validation_data=(x_test,y_test), shuffle=True, epochs=200, verbose=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 90, 100)           54800     
_________________________________________________________________
lambda_6 (Lambda)            (None, 14, 100)           0         
_________________________________________________________________
dense_7 (Dense)              (None, 14, 1)             101       
Total params: 54,901
Trainable params: 54,901
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 1273 samples, validate on 425 samples
Epoch 1/200
1273/1273 [==============================] - 4s 4ms/step - loss: 0.0497 - val_loss: 0.0361
Epoch 2/200
1273/1273 [==============================] - 3s 3ms/step - loss: 0.0385 - val_loss: 0.0366
Epoch 3/200
1273/1273 [==============================] - 3s 3ms/step - loss: 0.0370 - val_loss: 0.036

In [309]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
#From Sklearn documentation 
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = list(unique_labels(y_true, y_pred))
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [318]:
yhat = model.predict(x_test[0][np.newaxis,:,:])
print(yhat)


[[[ 0.00887503]
  [ 0.00156083]
  [-0.00268951]
  [-0.00519887]
  [ 0.00345523]
  [ 0.00361935]
  [ 0.00626405]
  [ 0.00337263]
  [ 0.00558755]
  [ 0.00440927]
  [ 0.00187594]
  [ 0.00257197]
  [ 0.00564536]
  [ 0.00561896]]]


In [ ]:
pred = model.predict(x_test[243][np.newaxis,:,:])
print(x_test[0].shape)
print(x_test.shape)

print(pred)
